# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_api_df = pd.read_csv("Output/weather.csv")
weather_api_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Port Victoria,KE,0.09,33.98,1594666349,58,66,71.78,1.10
1,Rikitea,PF,-23.12,-134.97,1594666350,56,84,72.88,15.17
2,Lompoc,US,34.64,-120.46,1594666289,3,72,69.01,8.05
3,Atuona,PF,-9.80,-139.03,1594666350,5,77,79.81,21.39
4,Mahbūbābād,IN,17.62,80.02,1594666350,100,77,81.09,2.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_api_df[["Latitude", "Longitude"]]
humidity = weather_api_df["Humidity"]
max_humidity = humidity.max()
locations.head()

,Latitude,Longitude
0,0.09,33.98
1,-23.12,-134.97
2,34.64,-120.46
3,-9.80,-139.03
4,17.62,80.02


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
figure_layout ={
    "width": "900px",
    "height": "600px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
vacation_spots = weather_api_df.loc[(weather_api_df["Max Temp"] > 70) & (weather_api_df["Max Temp"] < 80) & (weather_api_df["Cloudiness"] ==0), :]
vacation_spots = vacation_spots.dropna(how="any")
vacation_spots = vacation_spots.reset_index()
del vacation_spots ["index"]
vacation_spots.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Oktyabr'skiy,RU,54.48,53.47,1594666032,0,89,73.17,5.32
1,Cocentaina,ES,38.74,-0.44,1594666369,0,53,79.00,6.33
2,Villaviciosa,ES,43.48,-5.44,1594666385,0,66,73.00,5.06
3,Abu Samrah,SY,35.30,37.18,1594666388,0,59,76.84,11.99
4,Labuhan,ID,-6.88,112.21,1594665749,0,85,76.57,7.45


In [7]:
hotel_df = weather_api_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,Port Victoria,KE,0.09,33.98,1594666349,58,66,71.78,1.10,
1,Rikitea,PF,-23.12,-134.97,1594666350,56,84,72.88,15.17,
2,Lompoc,US,34.64,-120.46,1594666289,3,72,69.01,8.05,
3,Atuona,PF,-9.80,-139.03,1594666350,5,77,79.81,21.39,
4,Mahbūbābād,IN,17.62,80.02,1594666350,100,77,81.09,2.30,


In [10]:
hotel_info = []

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    params= {
        "location": f"{Latitude},{Longitude}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_info = requests.get(url, params=params)
    hotel_info = hotel_info.json()

NameError: name 'Latitude' is not defined

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map

figure_layout ={
    "width": "900px",
    "height": "600px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))


# Display figure
hotel_layer = gmaps.heatmap_layer(locations, weights=hotel_info, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(hotel_layer)
fig

TraitError: 
<dl>
<dt>Name</dt><dd></dd>
<dt>City</dt><dd>Port Victoria</dd>
<dt>Country</dt><dd>KE</dd>
</dl>
 is not a valid weight. Weights must be floats